In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("online_retail_II.csv",parse_dates=['InvoiceDate'])
df.rename(columns={'Customer ID':'CustomerID'},inplace=True)

In [ ]:
df

# Data Quality Analysis

In [ ]:
df.describe()

- Ada Quantity dan Price yang negative
- Ada banyak CustomerID yang missing
- Ada quantity hingga ribuan

In [ ]:
df['Quantity'].plot(kind='box')

In [ ]:
df[df['Quantity']<0].sample(10)

Quantity Negative terjadi pada :
- Invoice yang depannya C, yang berarti cancelled
- Invoice tanpa CustomerID

In [ ]:
df.sort_values('Quantity').tail(5)

In [ ]:
df.query('Quantity>0')['Quantity'].describe(percentiles=[0.5,0.8,0.9,0.95,0.99,0.999])

Bahkan top 99.9% transaksi adalah 50 item. Ratusan item disini lumayan mencurigakan dan akan kita exclude di analysis nantinya

In [ ]:
df.sort_values('Price').tail(5)

Description Manual menunjukkan transaksi yang bukan belanja

In [ ]:
df.query('Quantity>0').sort_values('Price').tail(5)

CustomerID null menunjukkan transaksi selain pembelian, lebih baik nantinya kita buang dari data

In [ ]:
drop_stock_code = ['POST','DOT','ADJUST','ADJUST2','M','B','AMAZONFEE']
df[~df['StockCode'].isin(drop_stock_code)].dropna(subset=['CustomerID']).query('Quantity>0').sort_values('Price')

# Data Cleansing

In [ ]:
df.dropna(subset=['CustomerID'],inplace=True) # Membuang CustomerID yang null

df = df[df['Quantity']<=df['Quantity'].quantile(0.99)] # Membuang Quantity yang terlalu ekstrim

df = df[df['Invoice'].str[0]!="C"] # Membuang invoice yang batal

df = df[df['Quantity']>0] # Mengambil quantity yang tag negative
df = df[~df['StockCode'].isin(drop_stock_code)] # Membuang beberapa code yang tidak terkait belanja
df['total_payment'] = df['Quantity']*df['Price']

# EDA

In [ ]:
df

In [ ]:
df.groupby('Country').agg({'Invoice':'nunique','total_payment':'sum','CustomerID':'nunique'}).sort_values('Invoice',ascending=False)

In [ ]:
df

In [ ]:
df['InvoiceTimeStamp'] = df['InvoiceDate'].copy()
df['InvoiceDate'] = df['InvoiceDate'].dt.normalize() # Mereset jam

In [ ]:
df.groupby('InvoiceDate')['Invoice'].nunique().plot()

In [ ]:
df.groupby('InvoiceDate')['total_payment'].sum().plot()

Pertanyaan yang harus dijawab :
- Apakah kita melakukan RFM segmentation ke seluruh customer? Apakah segmentasi perlu dibedakan antar negara?

In [ ]:
# Mari fokus ke customer dari United Kingdom saja
df = df[df['Country']=='United Kingdom']

# Melakukan Analysis RFM

# Recency Calculation

In [ ]:
df.sort_values('InvoiceDate').groupby('CustomerID').tail(1)

In [ ]:
# Recency
latest_date = df['InvoiceDate'].max()
latest_trx_per_customer = df.sort_values('InvoiceDate').groupby('CustomerID').tail(1)
latest_trx_per_customer['days_to_today'] = latest_date - latest_trx_per_customer['InvoiceDate']
latest_trx_per_customer['days_to_today']

In [ ]:
latest_trx_per_customer['days_to_today'] = latest_trx_per_customer['days_to_today'].dt.days # mengubah timedelta menjadi integer

In [ ]:
df_recency = latest_trx_per_customer[['CustomerID','days_to_today']]
df_recency['days_to_today'].plot(kind='hist')

In [ ]:
df_recency['R_score'] = pd.qcut(df_recency['days_to_today'],3,labels=['Recent','Normal','Long'])

In [ ]:
df_recency

In [ ]:
df_recency.groupby('R_score').agg({'days_to_today':['min','max','mean','median','count']})

# Volume Calculation

In [ ]:
df_volume = df.groupby('CustomerID').agg({'total_payment':'sum'})
df_volume['total_payment'].plot(kind='hist',bins=100)


In [ ]:
df_volume['V_score'] = pd.qcut(df_volume['total_payment'],3,labels=['Low','Medium','High'])

In [ ]:
df_volume

In [ ]:
df_volume.groupby('V_score').agg({'total_payment':['min','max','mean','median','count']})

# Frequency

In [ ]:
df_frequency = df.groupby('CustomerID').agg({'Invoice':'nunique'})
df_frequency['Invoice'].plot(kind='hist',bins=100)

In [ ]:
df_frequency['F_score'] = pd.qcut(df_frequency['Invoice'],3,labels=['Low','Medium','High'])

In [ ]:
df_frequency.groupby('F_score').agg({'Invoice':['min','max','mean','median','count']})

In [ ]:
import numpy as np
df_frequency['F_score'] = pd.cut(df_frequency['Invoice'],bins=[0,5,25,np.Inf],labels=['Low','Medium','High'],right=True)

In [ ]:
df_frequency['F_score'].value_counts()

In [ ]:
df_frequency.groupby('F_score').agg({'Invoice':['min','max','mean','median','count']})

# Penggabungan ketiga komponen

In [ ]:
df_frequency.head(2)

In [ ]:
df_recency.head(2)

In [ ]:
df_volume.head(2)

In [ ]:
df_recency.set_index('CustomerID')['R_score']

In [ ]:
recency = df_recency.set_index('CustomerID')['R_score']
volume = df_volume['V_score']
frequency = df_frequency['F_score']

In [ ]:
df_rfm = pd.concat([recency,volume,frequency],axis=1)

In [ ]:
df_rfm

Contoh pembagian berdasarkan RFM Analysis
1. High value Customers
2. Mid value Customers
3. Low value Customers
4. Dormant Customers

Contoh lain pembagian RFM :
- Top Customer (customers with high scores for all three parameters)
- Newcomers (customers with high recency but low frequency and monetary scores)
- at-risk customers (customers with low scores for all three parameters)

In [ ]:
df_rfm.value_counts()

# Challenge

Tim marketing ingin "membangkitkan" customer customer yang "Big dormant".

Big dormant adalah customer dengan : 
- R_score : Long
- V_score : Medium or High
- F_score : Medium or High  
Jika V_score dan F_score keduanya Medium maka tidak termasuk ke kategori "Big Dormant"  

Jika campaign yang mereka buat sukses membuat customer "Big dormant" ber transaksi seperti biasa, berapa total transaksi yang akan dihasilkan selama 1 bulan kedepan.